In [1]:
import clingo

from typing import Sequence

In [2]:
def solve(programs: Sequence[str], grounding_context=None, sep:str=' ', report_answersets=True):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            assert isinstance(model, clingo.Model)
            symbols = model.symbols(shown=True)
            if report_answersets:
                print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        mode = "UNKNOWN"
        solve_result = solve_handle.get()
        if solve_result.satisfiable:
            mode = "SAT"
        elif solve_result.unsatisfiable:
            mode = "UNSAT"
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(mode, "{}{}".format(len(models), cardinality_suffix))

    return models

In [3]:
def check(programs: Sequence[str], consistency_checks:str, grounding_context=None):
    print("Run without checks:")
    wo_checks = solve(programs, grounding_context, report_answersets=False)
    print("Run with checks:")
    w_checks = solve((*programs, consistency_checks), grounding_context, report_answersets=False)
    return len(wo_checks) == len(w_checks)

In [4]:
resources = """

resource("Geld").
resource("Essen").
resource("MB").
resource("MBP1").
resource("MBP2").
resource("MGP1").
resource("MGP2").
resource("MGP3").
resource("MV").

"""
solve([resources], report_answersets=False);

SAT 1


In [5]:
accounts = """
account("Besitzer:Maschinen Bauteile").
account("Besitzer:Maschine Bau Phase 1").
account("Besitzer:Maschine Bau Phase 2").
account("Besitzer:Maschine Gebrauch Phase 1").
account("Besitzer:Maschine Gebrauch Phase 2").
account("Besitzer:Maschine Gebrauch Phase 3").
account("Besitzer:Maschine Verbraucht").

account("Besitzer:Geld").
account("Besitzer:Essen").

account("Arbeiter:Geld").
account("Arbeiter:Essen").

"""
solve([accounts], report_answersets=False);

SAT 1


In [6]:
account_resource = """

account_resource("Besitzer:Maschinen Bauteile", "MB").
account_resource("Besitzer:Maschine Bau Phase 1", "MBP1").
account_resource("Besitzer:Maschine Bau Phase 2", "MBP2").
account_resource("Besitzer:Maschine Gebrauch Phase 1", "MGP1").
account_resource("Besitzer:Maschine Gebrauch Phase 2", "MGP2").
account_resource("Besitzer:Maschine Gebrauch Phase 3", "MGP3").
account_resource("Besitzer:Maschine Verbraucht", "MV").


account_resource("Arbeiter:Geld", "Geld").
account_resource("Arbeiter:Essen", "Essen").

"""
solve([account_resource], report_answersets=False);

SAT 1


In [7]:
agents = """

agent("Arbeiter").
agent("Besitzer").

"""
solve([agents], report_answersets=False);

SAT 1


In [8]:
items = """

item("Arbeitszeit").
item(amount_resource(1..2, "Essen")).
item(amount_resource(1..2, "Geld")).

"""
solve([items], report_answersets=False);

SAT 1


In [9]:
attainment = """

attainment(action("arbeiten")).
attainment(amount_resource(1..2, "Essen")).
attainment(amount_resource(1..2, "Geld")).

"""
solve([attainment], report_answersets=False);

SAT 1


In [10]:
actions = """

action("arbeiten").

action(provide_to_because(Attainment, Agent, Component)) :-
  attainment(Attainment), agent(Agent), component(Component),
  item(Item),
  component_item(Component, Item),
  component_provider(Component, Agent),
  item_attainment(Item, Attainment).

action(allow_from_because(Attainment, Agent, Component)) :-
  attainment(Attainment), agent(Agent), component(Component),
  item(Item),
  component_item(Component, Item),
  component_beneficiary(Component, Agent),
  item_attainment(Item, Attainment).

action(give_fromAcc_to_because(Attainment, Account, Agent, Component)) :-
  attainment(Attainment), account(Account), agent(Agent), component(Component),
  item(Item),
  component_item(Component, Item),
  component_provider(Component, Agent),
  item_attainment(Item, Attainment).

action(accept_intoAcc_from_because(Attainment, Account, Agent, Component)) :-
  attainment(Attainment), account(Account), agent(Agent), component(Component),
  item(Item),
  component_item(Component, Item),
  component_beneficiary(Component, Agent),
  item_attainment(Item, Attainment).

"""
solve([actions], report_answersets=False);

SAT 1


<block>:6:3-25: info: atom does not occur in any rule head:
  attainment(Attainment)

<block>:6:27-39: info: atom does not occur in any rule head:
  agent(Agent)

<block>:6:41-61: info: atom does not occur in any rule head:
  component(Component)

<block>:7:3-13: info: atom does not occur in any rule head:
  item(Item)

<block>:8:3-34: info: atom does not occur in any rule head:
  component_item(Component,Item)

<block>:9:3-39: info: atom does not occur in any rule head:
  component_provider(Component,Agent)

<block>:10:3-36: info: atom does not occur in any rule head:
  item_attainment(Item,Attainment)

<block>:13:3-25: info: atom does not occur in any rule head:
  attainment(Attainment)

<block>:13:27-39: info: atom does not occur in any rule head:
  agent(Agent)

<block>:13:41-61: info: atom does not occur in any rule head:
  component(Component)

<block>:14:3-13: info: atom does not occur in any rule head:
  item(Item)

<block>:15:3-34: info: atom does not occur in any rule head:
 

In [11]:
action_consequences = """

occ_at(agent_does(Provider, Action), T) :-
  agent(Provider), action(Action), time(T),
  agent(Beneficiary), component(Component),
  occ_at(agent_does(Provider, provide_to_because(action(Action), Beneficiary, Component)), T),
  occ_at(agent_does(Beneficiary, allow_from_because(action(Action), Provider, Component)), T).

occ_at(transaction(N, Res, From, To, Because), T) :-
  resource(Res), account(From), account(To), time(T),
  occ_at(agent_does(Provider, give_fromAcc_to_because(amount_resource(N, Res), From, Beneficiary, Because)), T),
  occ_at(agent_does(Beneficiary, accept_intoAcc_from_because(amount_resource(N, Res), To, Provider, Because)), T).

occ_at(income(1, "MBP1", "Besitzer:Maschine Bau Phase 1"), T) :-
  obs_at(acc_bal("Besitzer:Maschinen Bauteile", 2), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(expense(2, "Maschinen Bauteile", "Besitzer:Maschinen Bauteile"), T) :-
  obs_at(acc_bal("Besitzer:Maschinen Bauteile", 2), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(income(1, "MBP2", "Besitzer:Maschine Bau Phase 2"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Bau Phase 1", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(income(3, "Essen", "Besitzer:Essen"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Gebrauch Phase 1", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(expense(1, "MBP1", "Besitzer:Maschine Bau Phase 1"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Bau Phase 1", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(income(1, "MGP1", "Besitzer:Maschine Gebrauch Phase 1"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Bau Phase 2", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(expense(1, "MBP2", "Besitzer:Maschine Bau Phase 2"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Bau Phase 2", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(income(1, "MGP2", "Besitzer:Maschine Gebrauch Phase 2"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Gebrauch Phase 1", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(expense(1, "MBP1", "Besitzer:Maschine Gebrauch Phase 1"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Gebrauch Phase 1", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(income(1, "MGP3", "Besitzer:Maschine Gebrauch Phase 3"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Gebrauch Phase 2", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(expense(1, "MBP2", "Besitzer:Maschine Gebrauch Phase 2"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Gebrauch Phase 2", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(income(1, "MV", "Besitzer:Maschine Verbraucht"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Gebrauch Phase 3", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(expense(1, "MBP3", "Besitzer:Maschine Gebrauch Phase 3"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Gebrauch Phase 3", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(income(2, "MB", "Besitzer:Maschinen Bauteile"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Verbraucht", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(expense(1, "MV", "Besitzer:Maschine Verbraucht"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Verbraucht", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).


occ_at(income(3, "Essen", "Besitzer:Essen"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Bau Phase 2", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(income(3, "Essen", "Besitzer:Essen"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Gebrauch Phase 1", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(income(3, "Essen", "Besitzer:Essen"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Gebrauch Phase 2", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).

occ_at(income(3, "Essen", "Besitzer:Essen"), T) :-
  obs_at(acc_bal("Besitzer:Maschine Gebrauch Phase 3", 1), T),
  occ_at(agent_does("Arbeiter", "arbeiten"), T).


"""
solve([action_consequences], report_answersets=False);

SAT 1


<block>:4:3-18: info: atom does not occur in any rule head:
  agent(Provider)

<block>:4:20-34: info: atom does not occur in any rule head:
  action(Action)

<block>:4:36-43: info: atom does not occur in any rule head:
  time(T)

<block>:5:3-21: info: atom does not occur in any rule head:
  agent(Beneficiary)

<block>:5:23-43: info: atom does not occur in any rule head:
  component(Component)

<block>:10:3-16: info: atom does not occur in any rule head:
  resource(Res)

<block>:10:18-31: info: atom does not occur in any rule head:
  account(From)

<block>:10:33-44: info: atom does not occur in any rule head:
  account(To)

<block>:10:46-53: info: atom does not occur in any rule head:
  time(T)

<block>:15:3-55: info: atom does not occur in any rule head:
  obs_at(acc_bal("Besitzer:Maschinen Bauteile",2),T)

<block>:19:3-55: info: atom does not occur in any rule head:
  obs_at(acc_bal("Besitzer:Maschinen Bauteile",2),T)

<block>:23:3-57: info: atom does not occur in any rule head:
  obs

In [12]:
balance_rules = """

occ_at(account_valueChange(Acc, Deposit + Withdrawn + Income + Expense), T) :-
  account(Acc), time(T),
  Income = #sum { N,Res : occ_at(income(N, Res, Acc), T) },
  Expense = #sum { -N,Res : occ_at(expense(N, Res, Acc), T) },
  Withdrawn = #sum { -N,Res,Because : occ_at(transaction(N, Res, Acc, _, Because), T) },
  Deposit = #sum { N,Res,Because : occ_at(transaction(N, Res, _, Acc, Because), T) }.

obs_at(acc_bal(Acc, OldBal + Change), T+1) :-
  account(Acc), time(T+1),
  time(T),
  obs_at(acc_bal(Acc, OldBal), T),
  occ_at(account_valueChange(Acc, Change), T).

"""

In [13]:
balance_init = """

obs_at(acc_bal(Acc, 0), ST) :-
  time(ST), not time(ST - 1),
  account(Acc),
  not acc_initBal(Acc, _).
obs_at(acc_bal(Acc, I), ST) :-
  time(ST), not time(ST - 1),
  account(Acc),
  acc_initBal(Acc, I).

"""
solve([balance_init], report_answersets=False);

SAT 1


<block>:4:3-11: info: atom does not occur in any rule head:
  time(ST)

<block>:4:17-29: info: atom does not occur in any rule head:
  time((ST+-1))

<block>:5:3-15: info: atom does not occur in any rule head:
  account(Acc)

<block>:6:7-26: info: atom does not occur in any rule head:
  acc_initBal(#X0,#P1)

<block>:8:3-11: info: atom does not occur in any rule head:
  time(ST)

<block>:8:17-29: info: atom does not occur in any rule head:
  time((ST+-1))

<block>:9:3-15: info: atom does not occur in any rule head:
  account(Acc)

<block>:10:3-22: info: atom does not occur in any rule head:
  acc_initBal(Acc,I)



In [14]:
contract_domain = """

item_attainment("Arbeitszeit", action("arbeiten")).
item_attainment(amount_resource(N, "Essen"), amount_resource(N, "Essen")) :- N=1..2.
item_attainment(amount_resource(N, "Geld"), amount_resource(N, "Geld")) :- N=1..2.

contract(name_day("Dienstleistung", T)) :- T=1..7.

component(name_day("Arbeit", T)) :- T=1..7.
component(name_day("Lohn", T)) :- T=1..7.

contract_component(name_day("Dienstleistung", T), name_day("Arbeit", T)) :- T=1..7.
contract_component(name_day("Dienstleistung", T), name_day("Lohn", T)) :- T=1..7.

component_item(name_day("Arbeit", T), "Arbeitszeit") :- T=1..7.
component_item(name_day("Lohn", T), amount_resource(1, "Geld")) :- T=1..7.

component_provider(name_day("Arbeit", T), "Arbeiter") :- T=1..7.
component_beneficiary(name_day("Arbeit", T), "Besitzer") :- T=1..7.

component_provider(name_day("Lohn", T), "Besitzer") :- T=1..7.
component_beneficiary(name_day("Lohn", T), "Arbeiter") :- T=1..7.

contract(name_day("Handel", T)) :- T=4..7.

component(name_day("Lieferung", T)) :- T=4..7.
component(name_day("Bezahlung", T)) :- T=4..7.

contract_component(name_day("Handel", T), name_day("Lieferung", T)) :- T=4..7.
contract_component(name_day("Handel", T), name_day("Bezahlung", T)) :- T=4..7.

component_item(name_day("Lieferung", T), amount_resource(2, "Essen")) :- T=4..6.
component_item(name_day("Bezahlung", T), amount_resource(2, "Geld")) :- T=4..6.

component_item(name_day("Lieferung", 7), amount_resource(1, "Essen")).
component_item(name_day("Bezahlung", 7), amount_resource(1, "Geld")).

component_provider(name_day("Lieferung", T), "Besitzer") :- T=4..7.
component_beneficiary(name_day("Lieferung", T), "Arbeiter") :- T=4..7.
component_provider(name_day("Bezahlung", T), "Arbeiter") :- T=4..7.
component_beneficiary(name_day("Bezahlung", T), "Besitzer") :- T=4..7.

"""
solve([contract_domain], report_answersets=False);

SAT 1


In [15]:
contract_static = """

componentState("Initialisiert").
componentState("Gewährleistung").
componentState("Initialisiert [GLE]").
componentState("Gewährleistung [GLE]").

transitionAction("GLE").
transitionAction("LE").

state_transAction_state("Initialisiert","GLE","Initialisiert [GLE]").
state_transAction_state("Initialisiert","LE","Gewährleistung").

state_transAction_state("Initialisiert [GLE]","LE","Gewährleistung [GLE]").

state_transAction_state("Gewährleistung","GLE","Gewährleistung [GLE]").

"""
solve([contract_static], report_answersets=False);

SAT 1


In [16]:
contract_dynamic = """

component_dual(Component, Dual) :-
 component(Component), component(Dual),
 contract(Contract),
 Component != Dual,
 contract_component(Contract, Component), contract_component(Contract, Dual).

% If the transition event does not occur then there is no transition
-transComp_state_at(Component, SuccState, T) :-
  component(Component), componentState(SuccState), time(T),
  componentState(State), transitionAction(TransAction),
  deo_at(component_state(Component, State), T),
  state_transAction_state(State, TransAction, SuccState),
  not trans_comp_at(TransAction, Component, T).

% If two states have the same successor state and both states can be reached they will not transition
-transComp_state_at(Component, State1, T) :-
  component(Component), componentState(State1), time(T),
  componentState(State), componentState(State2), transitionAction(TransAction1), transitionAction(TransAction2),
  deo_at(component_state(Component, State), T),
  state_transAction_state(State, TransAction1, State1),
  state_transAction_state(State, TransAction2, State2),
  state_transAction_state(State1, TransAction2, SuccState),
  state_transAction_state(State2, TransAction1, SuccState),
  trans_comp_at(TransAction1, Component, T),
  trans_comp_at(TransAction2, Component, T).

% If a transition is not impossible then it will be executed
transComp_state_at(Component, SuccState, T) :-
  component(Component), componentState(SuccState), time(T),
  componentState(State),
  deo_at(component_state(Component, State), T),
  state_transAction_state(State, _, SuccState),
  not -transComp_state_at(Component, SuccState, T).

% INERTIA
deo_at(component_state(Component, State), T+1) :-
  component(Component), componentState(State), time(T+1),
  time(T),
  deo_at(component_state(Component, State), T),
  not transComp_state_at(Component, _, T).

% TRANSITION
deo_at(component_state(Component, SuccState), T+1) :-
  component(Component), componentState(SuccState), time(T+1),
  time(T), componentState(State),
  deo_at(component_state(Component, State), T),
  transComp_state_at(Component, SuccState, T).

%%%
trans_comp_at("LE", Component, T) :-
  component(Component), time(T),
  item(Item), attainment(Attainment), agent(Provider), agent(Beneficiary),
  component_provider(Component, Provider),
  component_beneficiary(Component, Beneficiary),
  component_item(Component, Item),
  item_attainment(Item, Attainment),
  occ_at(agent_does(Provider, give_fromAcc_to_because(Attainment, _, Beneficiary, Component)), T),
  occ_at(agent_does(Beneficiary, accept_intoAcc_from_because(Attainment, _, Provider, Component)), T).

trans_comp_at("LE", Component, T) :-
  component(Component), time(T),
  item(Item), attainment(Attainment), agent(Provider), agent(Beneficiary),
  component_provider(Component, Provider),
  component_beneficiary(Component, Beneficiary),
  component_item(Component, Item),
  item_attainment(Item, Attainment),
  occ_at(agent_does(Provider, provide_to_because(Attainment, Beneficiary, Component)), T),
  occ_at(agent_does(Beneficiary, allow_from_because(Attainment, Provider, Component)), T).

trans_comp_at("GLE", Component, T) :-
  component(Component), time(T),
  component(Dual),
  component_dual(Component, Dual),
  trans_comp_at("LE", Dual, T).

"""
solve([contract_dynamic], report_answersets=False);

SAT 1


<block>:4:2-22: info: atom does not occur in any rule head:
  component(Component)

<block>:4:24-39: info: atom does not occur in any rule head:
  component(Dual)

<block>:5:2-20: info: atom does not occur in any rule head:
  contract(Contract)

<block>:7:2-41: info: atom does not occur in any rule head:
  contract_component(Contract,Component)

<block>:7:43-77: info: atom does not occur in any rule head:
  contract_component(Contract,Dual)

<block>:11:3-23: info: atom does not occur in any rule head:
  component(Component)

<block>:11:25-50: info: atom does not occur in any rule head:
  componentState(SuccState)

<block>:11:52-59: info: atom does not occur in any rule head:
  time(T)

<block>:12:3-24: info: atom does not occur in any rule head:
  componentState(State)

<block>:12:26-55: info: atom does not occur in any rule head:
  transitionAction(TransAction)

<block>:14:3-57: info: atom does not occur in any rule head:
  state_transAction_state(State,TransAction,SuccState)

<block>

In [17]:
contract_init = """

deo_at(component_state(Component, "Initialisiert"), T) :-
  component(Component), time(T),
  contract(Contract),
  contract_component(Contract, Component),
  initContract_at(Contract, T).

"""
solve([contract_init], report_answersets=False);

SAT 1


<block>:4:3-23: info: atom does not occur in any rule head:
  component(Component)

<block>:4:25-32: info: atom does not occur in any rule head:
  time(T)

<block>:5:3-21: info: atom does not occur in any rule head:
  contract(Contract)

<block>:6:3-42: info: atom does not occur in any rule head:
  contract_component(Contract,Component)

<block>:7:3-31: info: atom does not occur in any rule head:
  initContract_at(Contract,T)



In [18]:
instance = """

time(1..9).

acc_initBal("Besitzer:Maschinen Bauteile", 2).
initContract_at(Contract, 1) :- contract(Contract).

%% Dienstleistung

occ_at(agent_does("Arbeiter", provide_to_because(action("arbeiten"), "Besitzer", name_day("Arbeit", T))), T) :- T=1..7.
occ_at(agent_does("Besitzer", allow_from_because(action("arbeiten"), "Arbeiter", name_day("Arbeit", T))), T) :- T=1..7.

occ_at(agent_does("Besitzer", give_fromAcc_to_because(amount_resource(1, "Geld"), "Besitzer:Geld", "Arbeiter", name_day("Lohn", T))), 8) :- T=1..7.
occ_at(agent_does("Arbeiter", accept_intoAcc_from_because(amount_resource(1, "Geld"), "Arbeiter:Geld", "Besitzer", name_day("Lohn", T))), 8) :- T=1..7.

%% Handel

occ_at(agent_does("Besitzer", give_fromAcc_to_because(amount_resource(2, "Essen"), "Besitzer:Essen", "Arbeiter", name_day("Lieferung", T))), T) :- T=4..6.
occ_at(agent_does("Arbeiter", accept_intoAcc_from_because(amount_resource(2, "Essen"), "Arbeiter:Essen", "Besitzer", name_day("Lieferung", T))), T) :- T=4..6.

occ_at(agent_does("Besitzer", give_fromAcc_to_because(amount_resource(1, "Essen"), "Besitzer:Essen", "Arbeiter", name_day("Lieferung", 7))), 7).
occ_at(agent_does("Arbeiter", accept_intoAcc_from_because(amount_resource(1, "Essen"), "Arbeiter:Essen", "Besitzer", name_day("Lieferung", 7))), 7).

occ_at(agent_does("Arbeiter", give_fromAcc_to_because(amount_resource(2, "Geld"), "Arbeiter:Geld", "Besitzer", name_day("Bezahlung", T))), 8) :- T=4..6.
occ_at(agent_does("Besitzer", accept_intoAcc_from_because(amount_resource(2, "Geld"), "Besitzer:Geld", "Arbeiter", name_day("Bezahlung", T))), 8) :- T=4..6.

occ_at(agent_does("Arbeiter", give_fromAcc_to_because(amount_resource(1, "Geld"), "Arbeiter:Geld", "Besitzer", name_day("Bezahlung", 7))), 8).
occ_at(agent_does("Besitzer", accept_intoAcc_from_because(amount_resource(1, "Geld"), "Besitzer:Geld", "Arbeiter", name_day("Bezahlung", 7))), 8).

"""
solve([instance], report_answersets=False);

SAT 1


<block>:6:33-51: info: atom does not occur in any rule head:
  contract(Contract)



In [19]:
prg = (
    resources,
    accounts,
    agents,
    items,
    attainment,
    actions,
    account_resource,
    action_consequences,
    balance_rules,
    balance_init,
    contract_domain,
    contract_static,
    contract_dynamic,
    contract_init,
    instance,
)

In [20]:
solve((*prg,
       '#show obs_at/2.',
       '#show occ_at/2.',
       '#show deo_at/2.',
       ), sep='\n');

Answer 1: {
deo_at(component_state(name_day("Arbeit",1),"Gewährleistung"),2)
deo_at(component_state(name_day("Arbeit",1),"Gewährleistung"),3)
deo_at(component_state(name_day("Arbeit",1),"Gewährleistung"),4)
deo_at(component_state(name_day("Arbeit",1),"Gewährleistung"),5)
deo_at(component_state(name_day("Arbeit",1),"Gewährleistung"),6)
deo_at(component_state(name_day("Arbeit",1),"Gewährleistung"),7)
deo_at(component_state(name_day("Arbeit",1),"Gewährleistung"),8)
deo_at(component_state(name_day("Arbeit",1),"Gewährleistung [GLE]"),9)
deo_at(component_state(name_day("Arbeit",1),"Initialisiert"),1)
deo_at(component_state(name_day("Arbeit",2),"Gewährleistung"),3)
deo_at(component_state(name_day("Arbeit",2),"Gewährleistung"),4)
deo_at(component_state(name_day("Arbeit",2),"Gewährleistung"),5)
deo_at(component_state(name_day("Arbeit",2),"Gewährleistung"),6)
deo_at(component_state(name_day("Arbeit",2),"Gewährleistung"),7)
deo_at(component_state(name_day("Arbeit",2),"Gewährleistung"),8)
deo_at(c

In [21]:
consistency_checks = """
:- account_resource(Acc, Res), not resource(Res).
:- account_resource(Acc, Res), not account(Acc).
:- obs_at(acc_bal(Acc, B1), T), obs_at(acc_bal(Acc, B2), T), B1 != B2.
:- acc_initBal(Acc, B), not account(Acc).
:- acc_initBal(Acc, B), B < 0.
"""

In [22]:
check(prg, consistency_checks)

Run without checks:
SAT 1
Run with checks:
SAT 1


True